### **Import Library**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from prophet import Prophet
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
import logging # Untuk mengontrol pesan log Prophet
import plotly.express as px

# Matikan logger Prophet agar output tidak terlalu banyak
logging.getLogger('prophet').setLevel(logging.WARNING)

In [2]:
# pip install Prophet

### **Import Data Hasil Rules**

In [3]:
df = pd.read_csv('D:/kuliah/PA/sakinah-mart/data/rules2023.csv')
df

,Unnamed: 0,antecedents,consequents,support,confidence,lift
0,0,"frozenset({'HARMONY MELON', 'HARMONY LEMON', '...",frozenset({'HARMONY STRAWBERY'}),0.000113,1,1235.260000
1,1,"frozenset({'DELFI TOP TRIP/CHO 9GR', 'MOMOGI J...",frozenset({'MOMOGI CKL 6GR'}),0.000113,1,686.255556
2,2,"frozenset({'INDOMILK KID VNL 115ML', 'CLOUD 9 ...",frozenset({'CLOUD 9 CHOCO 15 4G'}),0.000113,1,571.879630
3,3,"frozenset({'CLOUD 9 VANILA 15 4G', 'INDOMILK K...",frozenset({'CLOUD 9 CHOCO 15 4G'}),0.000146,1,571.879630
4,4,"frozenset({'INDOMILK KID VNL 115ML', 'CLOUD 9 ...",frozenset({'CLOUD 9 CHOCO 15 4G'}),0.000146,1,571.879630
5,5,"frozenset({'EKONOMI E900K', 'NUVO SOAP YLOW 72...",frozenset({'NUVO SOAP BLUE 72G'}),0.000113,1,388.446541
6,6,"frozenset({'INDOMILK STR 190ML BTL', 'INDOMILK...",frozenset({'INDOMILK KID VNL 115ML'}),0.000113,1,348.943503
7,7,"frozenset({'TONGTJI JASMINE 5 S', 'INDOMIE GRG...",frozenset({'INDOMIE AYAM SPECIAL'}),0.000113,1,284.622120
8,8,"frozenset({'KRIPIK SGKG TURBO 180G', 'TWISTER ...",frozenset({'OREO SOFT CAKE 16GR'}),0.000113,1,236.639847
9,9,"frozenset({'INDOMILK KID VNL 115ML', 'TEH GELA...",frozenset({'INDOMILK KID CKL 115ML'}),0.000162,1,183.818452


In [4]:
# Pastikan semua elemen dalam kolom antecedents dan consequents adalah frozenset
df["antecedents"] = df["antecedents"].apply(lambda x: frozenset(eval(x)) if isinstance(x, str) else x)
df["consequents"] = df["consequents"].apply(lambda x: frozenset(eval(x)) if isinstance(x, str) else x)

# Mengambil semua item unik
all_items = set()
for col in ["antecedents", "consequents"]:
    for items in df[col]:
        all_items.update(items)

all_items

{'BIG BLUEBRRY 375ML',
 'CIPTADENT PG FRS 190ML',
 'CLOUD 9 CHOCO 15 4G',
 'CLOUD 9 VANILA 15 4G',
 'DELFI TOP TRIP/CHO 9GR',
 'EKONOMI E900K',
 'HARMONY LEMON',
 'HARMONY MELON',
 'HARMONY ORANGE',
 'HARMONY STRAWBERY',
 'HERS PROT SCM 60 S',
 'INDOMIE AYAM BAWANG',
 'INDOMIE AYAM SPECIAL',
 'INDOMIE GRG SPECIAL',
 'INDOMIE SOTO MIE',
 'INDOMILK KID CKL 115ML',
 'INDOMILK KID STR 115ML',
 'INDOMILK KID VNL 115ML',
 'INDOMILK STR 190ML BTL',
 'KRIPIK SGKG TURBO 180G',
 'MOMOGI CKL 6GR',
 'MOMOGI JGG BKR 6GR',
 'NUVO SOAP BLUE 72G',
 'NUVO SOAP YLOW 72G',
 'OREO SOFT CAKE 16GR',
 'PARAMEX SKT KEPALA',
 'PUCUK HARUM TEH 350ML',
 'ROMA KELAPA 300GR',
 'SUNSILK SHP BLK SHN 160ML',
 'TEH GELAS ORI 170ML',
 'TONGTJI JASMINE 5 S',
 'TWISTER MINI CPP 20GR',
 'WAFELLO COCONUT 37 5G',
 'WALLS POPULAIRE STR',
 'YA KOPI SPESIAL 60GR'}

### **Import Data Transaksi**

In [5]:
groceries = pd.read_excel('D:/kuliah/PA/sakinah-mart/data/final_data2023.xlsx')
groceries

,TANGGAL,NO TRANSAKSI,NAMA BARANG,QTY
0,2023-01-01,2301011000001,FORVITA MARG 200GR,2
1,2023-01-01,2301011000002,SASA SANTAN KLPA 65ML,3
2,2023-01-01,2301011000003,CHEERS 1500ML GREEN,1
3,2023-01-01,2301011000004,SUN KARA 65ML,1
4,2023-01-01,2301011000004,OREO PIKACHU 165 6GR,1
...,...,...,...,...
162975,2023-12-31,2312311020112,KONIDIN 4 S,2
162976,2023-12-31,2312311020113,GOLDA CAPPUCINO 200ML,1
162977,2023-12-31,2312311020113,FLORIDINA ORANGE 360ML,1
162978,2023-12-31,2312311020114,WALLS PP TRICO/48,1


### **Filter Data Transaksi**

In [6]:
df_filtered = groceries[groceries['NAMA BARANG'].isin(all_items)]
df_filtered 

,TANGGAL,NO TRANSAKSI,NAMA BARANG,QTY
9,2023-01-01,2301011000008,OREO SOFT CAKE 16GR,2
34,2023-01-01,2301011000018,PUCUK HARUM TEH 350ML,6
62,2023-01-01,2301011000034,ROMA KELAPA 300GR,1
96,2023-01-01,2301011000040,INDOMIE AYAM BAWANG,1
100,2023-01-01,2301011000041,YA KOPI SPESIAL 60GR,1
...,...,...,...,...
162895,2023-12-31,2312311020095,INDOMIE GRG SPECIAL,4
162902,2023-12-31,2312311020096,INDOMIE GRG SPECIAL,4
162944,2023-12-31,2312311020102,INDOMIE GRG SPECIAL,2
162946,2023-12-31,2312311020102,INDOMIE SOTO MIE,2


In [7]:
# Menampilkan jumlah unik nama barang
jumlah_nama_barang = df_filtered['NAMA BARANG'].nunique()
print("Jumlah nama barang unik:", jumlah_nama_barang)

# Menampilkan isi nama barang yang unik
nama_barang_unik = df_filtered['NAMA BARANG'].unique()
print("Nama barang unik:")
for nama in nama_barang_unik:
    print("-", nama)

Jumlah nama barang unik: 35
Nama barang unik:
- OREO SOFT CAKE 16GR
- PUCUK HARUM TEH 350ML
- ROMA KELAPA 300GR
- INDOMIE AYAM BAWANG
- YA KOPI SPESIAL 60GR
- INDOMILK KID CKL 115ML
- INDOMILK STR 190ML BTL
- EKONOMI E900K
- INDOMILK KID STR 115ML
- TWISTER MINI CPP 20GR
- INDOMIE GRG SPECIAL
- INDOMIE AYAM SPECIAL
- WALLS POPULAIRE STR
- MOMOGI CKL 6GR
- HARMONY ORANGE
- HARMONY LEMON
- NUVO SOAP BLUE 72G
- NUVO SOAP YLOW 72G
- TONGTJI JASMINE 5 S
- INDOMIE SOTO MIE
- MOMOGI JGG BKR 6GR
- PARAMEX SKT KEPALA
- KRIPIK SGKG TURBO 180G
- SUNSILK SHP BLK SHN 160ML
- BIG BLUEBRRY 375ML
- HARMONY STRAWBERY
- HERS PROT SCM 60 S
- TEH GELAS ORI 170ML
- DELFI TOP TRIP/CHO 9GR
- CLOUD 9 CHOCO 15 4G
- CLOUD 9 VANILA 15 4G
- HARMONY MELON
- CIPTADENT PG FRS 190ML
- WAFELLO COCONUT 37 5G
- INDOMILK KID VNL 115ML


### **Agregasi Data Mingguan**

In [8]:
# Agregasi jumlah QTY per tanggal dan nama barang
df_grouped = df_filtered.groupby(['TANGGAL', 'NAMA BARANG'], as_index=False)['QTY'].sum()

tanggal_range = pd.date_range(start='2023-01-01', end='2023-12-31')
all_items = df_filtered['NAMA BARANG'].unique()

# Buat kombinasi semua tanggal dan item
multi_index = pd.MultiIndex.from_product([tanggal_range, all_items], names=['TANGGAL', 'NAMA BARANG'])
all_combinations = pd.DataFrame(index=multi_index).reset_index()

# Gabungkan dengan hasil groupby
df_joined = pd.merge(all_combinations, df_grouped, on=['TANGGAL', 'NAMA BARANG'], how='left')
df_joined['QTY'] = df_joined['QTY'].fillna(0)

# Pastikan TANGGAL dalam datetime
df_joined['TANGGAL'] = pd.to_datetime(df_joined['TANGGAL'])

# Urutkan dulu datanya
df_joined = df_joined.sort_values('TANGGAL')

# Ambil tanggal paling awal
start_date = df_joined['TANGGAL'].min()

# Hitung minggu ke-n secara manual (tiap 7 hari)
df_joined['WEEK_NUMBER'] = ((df_joined['TANGGAL'] - start_date).dt.days // 7) + 1

# Agregasi berdasarkan minggu dan nama barang
df_weekly = df_joined.groupby(['WEEK_NUMBER', 'NAMA BARANG'], as_index=False)['QTY'].sum()

# (Opsional) Tambahkan kolom tanggal mulai minggu
df_weekly['TANGGAL'] = start_date + pd.to_timedelta((df_weekly['WEEK_NUMBER'] - 1) * 7, unit='D')

# Visualisasi
fig = px.line(df_weekly, x='TANGGAL', y='QTY', color='NAMA BARANG',
              title='Jumlah Penjualan per Minggu (Setiap 7 Hari)')
fig.update_layout(legend_title_text='Klik untuk tampilkan/sembunyikan item')
fig.show()

In [9]:
df_weekly

,WEEK_NUMBER,NAMA BARANG,QTY,TANGGAL
0,1,BIG BLUEBRRY 375ML,1.0,2023-01-01
1,1,CIPTADENT PG FRS 190ML,0.0,2023-01-01
2,1,CLOUD 9 CHOCO 15 4G,0.0,2023-01-01
3,1,CLOUD 9 VANILA 15 4G,0.0,2023-01-01
4,1,DELFI TOP TRIP/CHO 9GR,0.0,2023-01-01
...,...,...,...,...
1850,53,TONGTJI JASMINE 5 S,0.0,2023-12-31
1851,53,TWISTER MINI CPP 20GR,0.0,2023-12-31
1852,53,WAFELLO COCONUT 37 5G,1.0,2023-12-31
1853,53,WALLS POPULAIRE STR,0.0,2023-12-31


### **Pivot Data**

In [10]:
df_pivoted= df_weekly.pivot(index='TANGGAL', columns='NAMA BARANG', values='QTY')
df_pivoted.columns.name = None  # hilangkan nama kolom atas
df_pivoted = df_pivoted.reset_index() 
df_pivoted

,TANGGAL,BIG BLUEBRRY 375ML,CIPTADENT PG FRS 190ML,CLOUD 9 CHOCO 15 4G,CLOUD 9 VANILA 15 4G,DELFI TOP TRIP/CHO 9GR,EKONOMI E900K,HARMONY LEMON,HARMONY MELON,HARMONY ORANGE,...,PARAMEX SKT KEPALA,PUCUK HARUM TEH 350ML,ROMA KELAPA 300GR,SUNSILK SHP BLK SHN 160ML,TEH GELAS ORI 170ML,TONGTJI JASMINE 5 S,TWISTER MINI CPP 20GR,WAFELLO COCONUT 37 5G,WALLS POPULAIRE STR,YA KOPI SPESIAL 60GR
0,2023-01-01,1.0,0.0,0.0,0.0,0.0,6.0,3.0,0.0,3.0,...,3.0,65.0,11.0,1.0,2.0,7.0,6.0,0.0,20.0,4.0
1,2023-01-08,2.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,...,4.0,52.0,2.0,1.0,2.0,1.0,3.0,0.0,5.0,2.0
2,2023-01-15,4.0,0.0,0.0,0.0,4.0,2.0,3.0,0.0,0.0,...,0.0,41.0,5.0,1.0,0.0,0.0,1.0,0.0,8.0,3.0
3,2023-01-22,1.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,...,0.0,31.0,7.0,0.0,0.0,5.0,2.0,0.0,7.0,3.0
4,2023-01-29,1.0,0.0,5.0,6.0,7.0,5.0,1.0,0.0,4.0,...,2.0,42.0,11.0,3.0,0.0,4.0,4.0,0.0,6.0,6.0
5,2023-02-05,2.0,0.0,1.0,2.0,3.0,2.0,0.0,0.0,1.0,...,5.0,48.0,9.0,3.0,0.0,6.0,3.0,0.0,2.0,4.0
6,2023-02-12,0.0,0.0,5.0,2.0,4.0,3.0,0.0,0.0,0.0,...,2.0,33.0,6.0,0.0,0.0,8.0,5.0,0.0,5.0,5.0
7,2023-02-19,0.0,0.0,2.0,2.0,4.0,3.0,0.0,0.0,0.0,...,0.0,54.0,12.0,0.0,0.0,3.0,4.0,0.0,10.0,4.0
8,2023-02-26,1.0,0.0,1.0,2.0,0.0,3.0,0.0,0.0,0.0,...,6.0,41.0,5.0,3.0,0.0,10.0,4.0,0.0,10.0,3.0
9,2023-03-05,1.0,0.0,1.0,3.0,0.0,1.0,1.0,2.0,1.0,...,2.0,29.0,13.0,1.0,0.0,4.0,2.0,0.0,6.0,3.0


In [11]:
# 1. Pisahkan kolom tanggal dan kolom produk
produk_df = df_pivoted.drop(columns=["TANGGAL"])

# 2. Hitung jumlah 0 per kolom
zero_counts = (produk_df == 0).sum()

# 3. Ambil item yang jumlah 0-nya <= 5
filtered_items = zero_counts[zero_counts <= 5].index

# 4. Filter dataframe hanya dengan item yang lolos kriteria 0
filtered_df = produk_df[filtered_items]

# 5. Hitung total penjualan per item
total_sales = filtered_df.sum().sort_values(ascending=False).index

# 6. Ambil 15 item dengan total penjualan tertinggi
# top_3_items = total_sales.head(15).index

# 7. Final dataframe dengan 15 item terbaik
df_final = df_pivoted[["TANGGAL"] + total_sales.tolist()]

In [12]:
df_final.head()

,TANGGAL,INDOMIE GRG SPECIAL,PUCUK HARUM TEH 350ML,INDOMIE SOTO MIE,INDOMIE AYAM BAWANG,INDOMILK KID STR 115ML,WALLS POPULAIRE STR,INDOMIE AYAM SPECIAL,OREO SOFT CAKE 16GR,ROMA KELAPA 300GR,INDOMILK STR 190ML BTL,NUVO SOAP YLOW 72G,KRIPIK SGKG TURBO 180G,NUVO SOAP BLUE 72G,YA KOPI SPESIAL 60GR
0,2023-01-01,174.0,65.0,12.0,14.0,13.0,20.0,8.0,11.0,11.0,7.0,9.0,3.0,10.0,4.0
1,2023-01-08,133.0,52.0,18.0,5.0,1.0,5.0,12.0,12.0,2.0,2.0,1.0,1.0,4.0,2.0
2,2023-01-15,133.0,41.0,14.0,6.0,8.0,8.0,6.0,0.0,5.0,8.0,4.0,0.0,5.0,3.0
3,2023-01-22,123.0,31.0,10.0,26.0,6.0,7.0,0.0,4.0,7.0,1.0,1.0,0.0,0.0,3.0
4,2023-01-29,84.0,42.0,12.0,12.0,11.0,6.0,0.0,7.0,11.0,6.0,12.0,0.0,15.0,6.0


In [13]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   TANGGAL                 53 non-null     datetime64[ns]
 1   INDOMIE GRG SPECIAL     53 non-null     float64       
 2   PUCUK HARUM TEH 350ML   53 non-null     float64       
 3   INDOMIE SOTO MIE        53 non-null     float64       
 4   INDOMIE AYAM BAWANG     53 non-null     float64       
 5   INDOMILK KID STR 115ML  53 non-null     float64       
 6   WALLS POPULAIRE STR     53 non-null     float64       
 7   INDOMIE AYAM SPECIAL    53 non-null     float64       
 8   OREO SOFT CAKE 16GR     53 non-null     float64       
 9   ROMA KELAPA 300GR       53 non-null     float64       
 10  INDOMILK STR 190ML BTL  53 non-null     float64       
 11  NUVO SOAP YLOW 72G      53 non-null     float64       
 12  KRIPIK SGKG TURBO 180G  53 non-null     float64     

In [14]:
# Ubah kolom TANGGAL menjadi datetime
df_final["TANGGAL"] = pd.to_datetime(df_final["TANGGAL"])

# Ubah dari wide ke long format agar cocok untuk plotly express
df_long = df_final.melt(id_vars="TANGGAL", var_name="NAMA BARANG", value_name="QTY")

# Buat line chart
fig = px.line(
    df_long,
    x="TANGGAL",
    y="QTY",
    color="NAMA BARANG",
    title="Jumlah Penjualan per Minggu per Item",
    markers=True,
    labels={"TANGGAL": "Tanggal", "QTY": "Jumlah Terjual", "NAMA BARANG": "Nama Produk"},
    hover_data={"TANGGAL": True, "QTY": True, "NAMA BARANG": True}
)

# Layout tambahan
fig.update_layout(
    legend_title_text='Klik nama produk untuk tampilkan/sembunyikan',
    xaxis_title='Tanggal',
    yaxis_title='Jumlah Terjual',
    template='plotly_white',
    height=500,
    width=900
)

fig.show()


C:\Users\ASUS\AppData\Local\Temp\ipykernel_360\2313548066.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [15]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   TANGGAL                 53 non-null     datetime64[ns]
 1   INDOMIE GRG SPECIAL     53 non-null     float64       
 2   PUCUK HARUM TEH 350ML   53 non-null     float64       
 3   INDOMIE SOTO MIE        53 non-null     float64       
 4   INDOMIE AYAM BAWANG     53 non-null     float64       
 5   INDOMILK KID STR 115ML  53 non-null     float64       
 6   WALLS POPULAIRE STR     53 non-null     float64       
 7   INDOMIE AYAM SPECIAL    53 non-null     float64       
 8   OREO SOFT CAKE 16GR     53 non-null     float64       
 9   ROMA KELAPA 300GR       53 non-null     float64       
 10  INDOMILK STR 190ML BTL  53 non-null     float64       
 11  NUVO SOAP YLOW 72G      53 non-null     float64       
 12  KRIPIK SGKG TURBO 180G  53 non-null     float64     

In [16]:
# Mengubah kolom TANGGAL menjadi datetime dan menjadikannya index
df_final['TANGGAL'] = pd.to_datetime(df_final['TANGGAL'])
df_final= df_final.set_index('TANGGAL')
df_final = df_final.asfreq('W-SUN') 
df_final

C:\Users\ASUS\AppData\Local\Temp\ipykernel_360\1700217815.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,INDOMIE GRG SPECIAL,PUCUK HARUM TEH 350ML,INDOMIE SOTO MIE,INDOMIE AYAM BAWANG,INDOMILK KID STR 115ML,WALLS POPULAIRE STR,INDOMIE AYAM SPECIAL,OREO SOFT CAKE 16GR,ROMA KELAPA 300GR,INDOMILK STR 190ML BTL,NUVO SOAP YLOW 72G,KRIPIK SGKG TURBO 180G,NUVO SOAP BLUE 72G,YA KOPI SPESIAL 60GR
TANGGAL,,,,,,,,,,,,,,
2023-01-01,174.0,65.0,12.0,14.0,13.0,20.0,8.0,11.0,11.0,7.0,9.0,3.0,10.0,4.0
2023-01-08,133.0,52.0,18.0,5.0,1.0,5.0,12.0,12.0,2.0,2.0,1.0,1.0,4.0,2.0
2023-01-15,133.0,41.0,14.0,6.0,8.0,8.0,6.0,0.0,5.0,8.0,4.0,0.0,5.0,3.0
2023-01-22,123.0,31.0,10.0,26.0,6.0,7.0,0.0,4.0,7.0,1.0,1.0,0.0,0.0,3.0
2023-01-29,84.0,42.0,12.0,12.0,11.0,6.0,0.0,7.0,11.0,6.0,12.0,0.0,15.0,6.0
2023-02-05,101.0,48.0,15.0,2.0,6.0,2.0,3.0,9.0,9.0,3.0,6.0,4.0,4.0,4.0
2023-02-12,13.0,33.0,17.0,10.0,7.0,5.0,14.0,5.0,6.0,1.0,2.0,9.0,9.0,5.0
2023-02-19,47.0,54.0,25.0,9.0,5.0,10.0,10.0,5.0,12.0,2.0,4.0,7.0,5.0,4.0
2023-02-26,103.0,41.0,14.0,18.0,10.0,10.0,20.0,12.0,5.0,5.0,5.0,5.0,11.0,3.0


In [17]:
# 1. Mengubah index 'TANGGAL' menjadi kolom biasa
df_final = df_final.reset_index()
df_final.rename(columns={'index': 'TANGGAL'}, inplace=True) # Rename kolom 'index' yang baru dibuat menjadi 'TANGGAL'

# 2. Pastikan kolom 'TANGGAL' adalah tipe datetime (sekarang sudah jadi kolom biasa)
df_final['TANGGAL'] = pd.to_datetime(df_final['TANGGAL'])

# 3. Identifikasi kolom produk
product_columns = [col for col in df_final.columns if col != 'TANGGAL']

In [18]:
# --- Fungsi untuk Menghitung Metrik ---
def calculate_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    # Calculate MAPE, handling division by zero for actuals
    # Only include points where y_true is not zero for MAPE calculation
    non_zero_indices = y_true != 0
    if np.sum(non_zero_indices) > 0:
        mape = np.mean(np.abs((y_true[non_zero_indices] - y_pred[non_zero_indices]) / y_true[non_zero_indices])) * 100
    else:
        mape = np.nan # Jika semua nilai aktual adalah 0, MAPE tidak terdefinisi

    return mae, rmse, mape

In [19]:
# --- Persiapan untuk Forecasting ---
results = []
train_size = int(len(df_final) * 0.8)
test_size = len(df_final) - train_size

print("="*80)
print("             MEMULAI FORECASTING DENGAN PROPHET DAN EVALUASI             ")
print("="*80)
print(f"Total data historis: {len(df_final)} titik.")
print(f"Ukuran data training (80%): {train_size} titik.")
print(f"Ukuran data testing (20%): {test_size} titik.")
print("\n")

# Loop melalui setiap kolom produk untuk melakukan forecasting
for product in product_columns:
    print(f"\n--- Memproses Produk: {product} ---")

    # Siapkan DataFrame untuk Prophet: 'ds' (datetime) dan 'y' (nilai numerik)
    df_prophet = df_final[['TANGGAL', product]].copy()
    df_prophet.rename(columns={'TANGGAL': 'ds', product: 'y'}, inplace=True)

    # Membagi data menjadi training dan testing (split time series)
    train_df = df_prophet.iloc[:train_size]
    test_df = df_prophet.iloc[train_size:]

    # Inisialisasi dan latih model Prophet
    # Karena data adalah agregasi mingguan (Tanggal adalah awal minggu),
    # seasonality mingguan internal Prophet tidak relevan.
    # Yearly seasonality juga dimatikan karena hanya ~1 tahun data.
    m = Prophet(
        yearly_seasonality=False,
        weekly_seasonality=False,
        daily_seasonality=False,
        growth='linear',
        changepoint_prior_scale=0.05 # Default value
    )
    m.fit(train_df)

    # Buat DataFrame untuk periode waktu yang akan diprediksi (periode test)
    # Ini harus mencakup tanggal-tanggal yang ada di test_df
    future = m.make_future_dataframe(periods=len(test_df), freq='W', include_history=False) # Hanya future dates

    # Lakukan prediksi
    forecast = m.predict(future)

    # Ambil nilai prediksi untuk periode test
    y_true = test_df['y'].values
    y_pred = forecast['yhat'].values

    # Hitung metrik evaluasi
    mae, rmse, mape = calculate_metrics(y_true, y_pred)

    print(f"  - MAE: {mae:.2f}")
    print(f"  - RMSE: {rmse:.2f}")
    if pd.isna(mape):
        print(f"  - MAPE: N/A (Semua nilai aktual adalah 0 di periode testing)")
    else:
        print(f"  - MAPE: {mape:.2f}%")

    # Simpan hasil
    results.append({
        'Product': product,
        'MAE': mae,
        'RMSE': rmse,
        'MAPE': mape
    })

# Konversi hasil ke DataFrame untuk analisis lebih lanjut
df_results = pd.DataFrame(results)

print("\n")
print("="*80)
print("             RINGKASAN HASIL EVALUASI MODEL FORECASTING             ")
print("="*80)

# Urutkan berdasarkan MAPE (terendah ke tertinggi), lalu MAE
df_results_sorted = df_results.sort_values(by='MAPE', ascending=True)

print(df_results_sorted.to_string(index=False)) # to_string() untuk menampilkan semua baris tanpa truncasi

print("\n📈 Rata-rata Performa Semua Produk:")
print(f"Avg MAE: {df_results_sorted['MAE'].mean():.2f}")
print(f"Avg RMSE: {df_results_sorted['RMSE'].mean():.2f}")
print(f"Avg MAPE: {df_results_sorted['MAPE'].mean():.2f}%")

print("--- Catatan Penting ---")
print("1. Akurasi model mungkin terbatas karena jumlah data historis yang relatif sedikit (53 titik).")
print("2. 'MAPE: N/A' menunjukkan bahwa semua nilai aktual di periode testing adalah 0.")
print("3. Hasil forecast ini berdasarkan data dummy. Untuk hasil nyata, gunakan data aktual Anda.")

             MEMULAI FORECASTING DENGAN PROPHET DAN EVALUASI             
Total data historis: 53 titik.
Ukuran data training (80%): 42 titik.
Ukuran data testing (20%): 11 titik.



--- Memproses Produk: INDOMIE GRG SPECIAL ---


13:43:18 - cmdstanpy - INFO - Chain [1] start processing
13:43:19 - cmdstanpy - INFO - Chain [1] done processing


  - MAE: 36.21
  - RMSE: 42.77
  - MAPE: 41.48%

--- Memproses Produk: PUCUK HARUM TEH 350ML ---


13:43:19 - cmdstanpy - INFO - Chain [1] start processing
13:43:19 - cmdstanpy - INFO - Chain [1] done processing
13:43:19 - cmdstanpy - INFO - Chain [1] start processing


  - MAE: 19.30
  - RMSE: 24.82
  - MAPE: 221.83%

--- Memproses Produk: INDOMIE SOTO MIE ---


...

  - MAE: 1.92
  - RMSE: 2.24
  - MAPE: 97.88%

--- Memproses Produk: NUVO SOAP BLUE 72G ---


13:43:23 - cmdstanpy - INFO - Chain [1] start processing
13:43:24 - cmdstanpy - INFO - Chain [1] done processing
13:43:24 - cmdstanpy - INFO - Chain [1] start processing


  - MAE: 3.58
  - RMSE: 4.35
  - MAPE: 74.63%

--- Memproses Produk: YA KOPI SPESIAL 60GR ---


13:43:24 - cmdstanpy - INFO - Chain [1] done processing


  - MAE: 2.72
  - RMSE: 3.68
  - MAPE: 70.62%


             RINGKASAN HASIL EVALUASI MODEL FORECASTING             
               Product       MAE      RMSE       MAPE
   WALLS POPULAIRE STR  3.996129  5.558218  38.278510
  INDOMIE AYAM SPECIAL  4.641233  5.395978  41.471877
   INDOMIE GRG SPECIAL 36.205780 42.768864  41.481728
INDOMILK KID STR 115ML  7.301539  9.533726  49.888527
   INDOMIE AYAM BAWANG  4.717086  5.610380  51.231511
      INDOMIE SOTO MIE  7.214112  9.384575  57.752013
     ROMA KELAPA 300GR  2.632628  3.940891  66.463178
  YA KOPI SPESIAL 60GR  2.719650  3.675891  70.621866
    NUVO SOAP BLUE 72G  3.578971  4.345774  74.627713
    NUVO SOAP YLOW 72G  1.902192  2.285752  80.471019
INDOMILK STR 190ML BTL  4.121424  4.638709  89.505399
   OREO SOFT CAKE 16GR  3.988291  5.047764  97.730715
KRIPIK SGKG TURBO 180G  1.922825  2.243207  97.876867
 PUCUK HARUM TEH 350ML 19.300475 24.823143 221.831724

📈 Rata-rata Performa Semua Produk:
Avg MAE: 7.45
Avg RMSE: 9.23
Avg MAPE